In [1]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import pandas as pd
from sklearn import svm
import numpy as np
import math

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colors
from sklearn.model_selection import train_test_split
from sklearn import datasets
from matplotlib.colors import ListedColormap
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit,StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GridSearchCV, LeaveOneOut, cross_val_predict
from sklearn.model_selection import KFold  
from sklearn.linear_model import LogisticRegression  
from sklearn.naive_bayes import GaussianNB  
from sklearn.neighbors import KNeighborsClassifier   
from sklearn import svm  
from sklearn.tree import DecisionTreeClassifier  
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
import math
import datetime
import multiprocessing as mp
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC

import random


# In[25]:


# cores = mp.cpu_count()
# pool = mp.Pool(processes=20)

# min1=min(list(X_standard.shape))
# a1=list(range(10,min1,10))
# a1[-1]=min1
# pcc_top1=pool.map(find_best_pca_lr,a1)  ##（SVM，LR）
# pool.terminate()


# a1=pd.DataFrame()
# for i1 in pcc_top1:
#     a1=pd.concat([a1,i1], axis=0)
# a1.columns=['tr_TP', 'tr_TN', 'tr_FP', 'tr_FN', 'tr_Sen', 'tr_Spe', 'tr_Acc', 'tr_PPV', 'tr_NPV', 'tr_MCC', 'tr_AUC','TP',
#        'TN', 'FP', 'FN', 'Sen', 'Spe', 'Acc', 'PPV', 'NPV', 'MCC', 'AUC','PCA']   

# a1=a1.sort_values(['Acc','tr_Acc','PCA'], ascending=[False,False,True])

In [2]:
def read_single_csv(input_path,i):

    df_chunk=pd.read_csv(input_path,
                         chunksize=10000,
#                          sep='\t'
                        sep=i
                        )
    res_chunk=[]
    for chunk in df_chunk:
        res_chunk.append(chunk)
    res_df=pd.concat(res_chunk)
    return res_df

In [3]:
def get_measures_gridloo(label, score):
    label = np.array(label)
    score = np.array(score)
    
    N  = len(label)
    TP = sum((label == 1) & (score == 1))
    TN = sum((label == 0) & (score == 0))
    FP = sum((label == 0) & (score == 1))
    FN = sum((label == 1) & (score == 0))

    # init all measures to nan
    measures = {measure: float("nan") for measure in ("Sen", "Spe", "Acc", "PPV", "NPV", "MCC","AUC")}
    
    measures["TP"] = TP
    measures["TN"] = TN
    measures["FP"] = FP
    measures["FN"] = FN
    
    S = (TP + FN) / N
    P = (TP + FP) / N

    if (TP + FN) > 0:
        measures["Sen"] = round(TP/(TP+FN), 4)

    if (TN + FP) > 0:
        measures["Spe"] = round(TN/(TN+FP), 4)

    if (TP + FP + FN + TN) > 0:
        measures["Acc"] = round((TP+TN)/(TP+FP+FN+TN), 4)

    if (TP + FP) > 0:
        measures["PPV"] = round(TP/(TP+FP), 4)

    if (TN + FN) > 0:
        measures["NPV"] = round(TN/(TN+FN), 4)

    if (S*P*(1-S)*(1-P)) > 0:
        measures["MCC"] = round((TP/N - S*P)/(math.sqrt(S*P*(1-S)*(1-P))), 4)
    
    
    measures["AUC"]= roc_auc_score(label, score)
    return pd.DataFrame([measures],
                        columns=["TP", "TN", "FP",
                                 "FN", "Sen", "Spe", "Acc", "PPV", "NPV", "MCC","AUC"])

In [4]:
##SVM
def find_best_pca(i):
    n_components=i
    estimator = PCA(n_components=n_components,random_state=10)
    pca_X_train = estimator.fit_transform(X_standard)
    pca_X_test = estimator.transform(X_standard_test)
    
    scoring = {'AUC': 'roc_auc'}
    cost = [-5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
    gam = [3, 1, -1, -3, -5, -7, -9, -11, -13, -15]
    parameters =[{'kernel': ['rbf'], 'C': [2**x for x in cost],'gamma':[2**x for x in gam]}]
    
    cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 
    grid_search=GridSearchCV(estimator=SVC(random_state=10),param_grid=parameters,cv=cvx,scoring='accuracy')
    grid_search.fit(pca_X_train, train_y)
    
    train_pre_y = cross_val_predict(grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
    train_res1=get_measures_gridloo(train_y,train_pre_y)
#     train_res1['PCA']=n_components
    test_pre_y = grid_search.predict(pca_X_test)
    test_res1=get_measures_gridloo(test_y,test_pre_y)
    test_res1['PCA']=n_components
    
    res=pd.concat([train_res1,test_res1], axis=1)
    return res

In [5]:
def find_best_pca_lr(i):
    n_components=i
    estimator = PCA(n_components=n_components,random_state=10)
    pca_X_train = estimator.fit_transform(X_standard)
    pca_X_test = estimator.transform(X_standard_test)
    
    cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 

    param_grid = {'penalty':['l2'],
              "C":[0.00001,0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000],  
              "solver":["newton-cg", "lbfgs","liblinear"],
#             "C":list(np.arange(0,0.1,0.005)),   
#               "solver":["newton-cg", "lbfgs","liblinear"]
            }
    LR_grid = LogisticRegression()
    grid_search = GridSearchCV(LR_grid, param_grid=param_grid, cv=cvx ,scoring='accuracy',n_jobs=9)
    grid_search.fit(pca_X_train, train_y)


    train_pre_y = cross_val_predict(grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
    train_res1=get_measures_gridloo(train_y,train_pre_y)
#     train_res1['PCA']=n_components
   

    test_pre_y = grid_search.predict(pca_X_test)
    test_res1=get_measures_gridloo(test_y,test_pre_y)
    test_res1['PCA']=n_components
    
    res=pd.concat([train_res1,test_res1], axis=1)
    return res

In [9]:
cancer='HNSC'
path='D:\\lymph_meta_xmiseq\\tcga_data\\'
# e1="./"+cancer+"_delta_pcc_wilcox.csv"
# delta_pcc=read_single_csv(e1,',')
# delta_pcc=delta_pcc.drop(['Unnamed: 0','pair'],axis=1)
# delta_pcc=delta_pcc.dropna(axis=0,how='any')

# result=delta_pcc.T
# train=result.iloc[:,0:-1].values.astype('float')
# target=result.iloc[:,-1].values.astype('float')

train=pd.read_csv(path+cancer+"\\admat_final.csv").iloc[:,2:].T.values
target=pd.read_csv(path+cancer+"\\gctm_label.csv",index_col=0).values.ravel()

In [10]:
train

array([[ 8.61302310e-03, -1.67276214e-02, -7.71436140e-02, ...,
         1.49324996e-02, -9.55921388e-04,  1.64712682e-04],
       [-1.01694133e-02,  5.04230888e-02,  2.35664766e-01, ...,
         1.22894564e-02, -8.40306508e-03, -6.14264990e-02],
       [ 1.36782686e-02,  3.05819334e-02,  1.00714180e-02, ...,
        -2.37681151e-02, -9.66026000e-05, -8.76877865e-03],
       ...,
       [ 2.17807588e-03, -2.54476766e-03,  3.25767040e-03, ...,
         1.49324996e-02,  2.65327542e-03, -3.56168709e-04],
       [ 3.56776436e-03, -7.91398304e-04, -2.99311432e-02, ...,
         7.51862011e-03,  9.02637160e-03, -4.25091257e-03],
       [ 1.23089540e-02,  1.73426352e-02, -4.00936485e-02, ...,
         1.49324996e-02,  9.02637160e-03,  8.16723247e-03]])

In [11]:
target

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0], dtype=int64)

In [183]:
train_X,test_X, train_y, test_y = train_test_split(train,
                                                   target,
                                                   test_size = 0.3,
                                                   stratify=target,
                                                   random_state=10
                                                   )

standardScaler = StandardScaler()
standardScaler.fit(train_X)
X_standard = standardScaler.transform(train_X)
X_standard_test = standardScaler.transform(test_X)

n_components=0.95
estimator = PCA(n_components=n_components,random_state=10)
pca_X_train = estimator.fit_transform(X_standard)
pca_X_test = estimator.transform(X_standard_test)
cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 

In [184]:
train_X.shape

(126, 2281)

In [185]:
#svm
scoring = {'AUC': 'roc_auc'}
cost = [-5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
gam = [3, 1, -1, -3, -5, -7, -9, -11, -13, -15]
parameters =[{'kernel': ['rbf'], 'C': [2**x for x in cost],'gamma':[2**x for x in gam]}]
cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=10) 

svc_grid_search=GridSearchCV(estimator=SVC(random_state=10),param_grid=parameters,cv=cvx,scoring='accuracy')
svc_grid_search.fit(pca_X_train, train_y)

#LR
param_grid = {'penalty':['l1', 'l2'],
              "C":[0.00001,0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000],
              "solver":["newton-cg", "lbfgs","liblinear","sag","saga"]
#               "algorithm":['auto', 'ball_tree', 'kd_tree', 'brute']
            }
LR_grid = LogisticRegression(max_iter=1000)
LR_grid_search = GridSearchCV(LR_grid, param_grid=param_grid, cv=cvx ,scoring='accuracy',n_jobs=10)
LR_grid_search.fit(pca_X_train, train_y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.45230769        nan 0.52369231 0.54769231
 0.54769231 0.72153846 0.76123077 0.74523077        nan        nan
 0.45230769        nan 0.50030769 0.75292308 0.75292308 0.75292308
 0.79230769 0.75292308        nan        nan 0.45230769        nan
 0.52369231 0.848      0.848      0.82430769 0.864      0.832
        nan        nan 0.68246154        nan 0.75292308 0.888
 0.888      0.87230769 0.88       0.88              nan        nan
 0.92              nan 0.92       0.88061538 0.88061538 0.85661538
 0.86461538 0.86461538        nan        nan 0.944             nan
 0.936      0.84061538 0.84061538 0.86461538 0.86461538 0.87261538
        nan        nan 0.944             nan 0.87261538 0.84861538
 0.84861538 0.86461538 0.86461538 0.87261538        nan        nan
 0.90430769        nan 0.87261538 0.85661538 0.85661538 0.84861

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=10, shuffle=True),
             estimator=LogisticRegression(max_iter=1000), n_jobs=10,
             param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100,
                               1000],
                         'penalty': ['l1', 'l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             scoring='accuracy')

In [186]:
estimators = [
    ('lr', LR_grid_search.best_estimator_),
    ('svc', svc_grid_search.best_estimator_),
]
clf = StackingClassifier(estimators=estimators, final_estimator=LinearSVC(C=5),n_jobs=10)
clf.fit(pca_X_train, train_y)

StackingClassifier(estimators=[('lr',
                                LogisticRegression(C=1, max_iter=1000,
                                                   penalty='l1',
                                                   solver='liblinear')),
                               ('svc',
                                SVC(C=32, gamma=3.0517578125e-05,
                                    random_state=10))],
                   final_estimator=LinearSVC(C=5), n_jobs=10)

In [187]:
estimators = [
    ('lr', LR_grid_search.best_estimator_),
    ('svc', svc_grid_search.best_estimator_),
]

param_grid = {'final_estimator':[LogisticRegression(C=0.00001),LogisticRegression(C=0.0001),LogisticRegression(C=0.001),LogisticRegression(C=0.01),
                                 LogisticRegression(C=0.1),LogisticRegression(C=1),LogisticRegression(C=10),LogisticRegression(C=100),
                                 LogisticRegression(C=1000)]}

Stacking_grid =StackingClassifier(estimators=estimators,)
Stacking_grid_search = GridSearchCV(Stacking_grid, param_grid=param_grid, cv=cvx ,scoring='accuracy',n_jobs=10)
Stacking_grid_search.fit(pca_X_train, train_y)
Stacking_grid_search.best_estimator_

StackingClassifier(estimators=[('lr',
                                LogisticRegression(C=1, max_iter=1000,
                                                   penalty='l1',
                                                   solver='liblinear')),
                               ('svc',
                                SVC(C=32, gamma=3.0517578125e-05,
                                    random_state=10))],
                   final_estimator=LogisticRegression(C=100))

In [188]:
train_pre_y = cross_val_predict(Stacking_grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
train_res1=get_measures_gridloo(train_y,train_pre_y)
#     train_res1['PCA']=n_components

test_pre_y = Stacking_grid_search.predict(pca_X_test)
test_res1=get_measures_gridloo(test_y,test_pre_y)

In [189]:
print(train_res1)
print(test_res1)

   TP  TN  FP  FN     Sen     Spe     Acc     PPV     NPV     MCC       AUC
0  64  50   7   5  0.9275  0.8772  0.9048  0.9014  0.9091  0.8076  0.902365
   TP  TN  FP  FN  Sen  Spe  Acc  PPV  NPV  MCC  AUC
0  30  24   0   0  1.0  1.0  1.0  1.0  1.0  1.0  1.0


In [79]:
%%time
################random seed loop#################
train_scores=[]
test_scores=[]
loopn=10
scoring='roc_auc'
np.random.seed(42)
random_states=np.random.choice(range(101), loopn, replace=False)

tmp=[]
#print(random_states)
for i in range(loopn):
    train_X,test_X, train_y, test_y = train_test_split(train,
                                                   target,
                                                   test_size = 0.3,
                                                   stratify=target,
                                                   random_state=random_states[i]
                                                   )

    standardScaler = StandardScaler()
    standardScaler.fit(train_X)
    X_standard = standardScaler.transform(train_X)
    X_standard_test = standardScaler.transform(test_X)
    #calculate max n_components
    estimator = PCA(n_components=0.95,random_state=42)
    pca_X_train = estimator.fit_transform(X_standard)

    n_components=range(10,min(pca_X_train.shape),10)
    
    #n_components=[0.99,0.95,0.90,0.85]
    best_pca_train_scores=[]
    best_pca_test_scores=[]
    for j in n_components:
        estimator = PCA(n_components=j,random_state=42)
        pca_X_train = estimator.fit_transform(X_standard)
        pca_X_test = estimator.transform(X_standard_test)
        cvx = StratifiedKFold(n_splits=5, shuffle=True, random_state=42) 
    
        #scoring = {'AUC': 'roc_auc'}
        cost = [-5, -3, -1, 1, 3, 5, 7, 9, 11, 13, 15]
        gam = [3, 1, -1, -3, -5, -7, -9, -11, -13, -15]
        parameters =[{'kernel': ['rbf'], 'C': [2**x for x in cost],'gamma':[2**x for x in gam]}]

        svc_grid_search=GridSearchCV(estimator=SVC(random_state=42),
                                     param_grid=parameters,cv=cvx,scoring=scoring)
        svc_grid_search.fit(pca_X_train, train_y)

        param_grid = {'penalty':['l1', 'l2'],
                      "C":[0.00001,0.0001,0.001, 0.01, 0.1, 1, 10, 100, 1000],
                      "solver":["newton-cg", "lbfgs","liblinear","sag","saga"]
    #               "algorithm":['auto', 'ball_tree', 'kd_tree', 'brute']
                    }
        LR_grid = LogisticRegression(max_iter=1000)
        LR_grid_search = GridSearchCV(LR_grid, param_grid=param_grid, cv=cvx ,scoring=scoring,n_jobs=10)
        LR_grid_search.fit(pca_X_train, train_y)

        estimators = [
            ('lr', LR_grid_search.best_estimator_),
            ('svc', svc_grid_search.best_estimator_),
        ]
        clf = StackingClassifier(estimators=estimators, final_estimator=LinearSVC(C=5),n_jobs=10)
        clf.fit(pca_X_train, train_y)

        estimators = [
            ('lr', LR_grid_search.best_estimator_),
            ('svc', svc_grid_search.best_estimator_),
        ]

        param_grid = {'final_estimator':[LogisticRegression(C=0.00001),LogisticRegression(C=0.0001),
                                         LogisticRegression(C=0.001),LogisticRegression(C=0.01),
                                         LogisticRegression(C=0.1),LogisticRegression(C=1),
                                         LogisticRegression(C=10),LogisticRegression(C=100),
                                         LogisticRegression(C=1000)]}

        Stacking_grid =StackingClassifier(estimators=estimators,)
        Stacking_grid_search = GridSearchCV(Stacking_grid, param_grid=param_grid, cv=cvx,
                                            scoring=scoring,n_jobs=10)
        Stacking_grid_search.fit(pca_X_train, train_y)
        Stacking_grid_search.best_estimator_

        train_pre_y = cross_val_predict(Stacking_grid_search.best_estimator_, pca_X_train,train_y, cv=cvx)
        train_res1=get_measures_gridloo(train_y,train_pre_y)

        test_pre_y = Stacking_grid_search.predict(pca_X_test)
        test_res1=get_measures_gridloo(test_y,test_pre_y)
        
        best_pca_train_scores.append(train_res1.loc[:,"AUC"])
        best_pca_test_scores.append(test_res1.loc[:,"AUC"])
    
    train_scores.append(np.max(best_pca_train_scores))
    test_scores.append(best_pca_test_scores[np.argmax(best_pca_train_scores)])
    
    tmp.append(best_pca_train_scores)
    tmp.append(best_pca_test_scores)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.87351815
 0.87351815 0.87600733 0.87600733 0.87600733        nan        nan
 0.5               nan 0.5        0.88747919 0.88747919 0.89395604
 0.89395604 0.89395604        nan        nan 0.5               nan
 0.5        0.91307026 0.91307026 0.91699967 0.91568432 0.91699967
        nan        nan 0.87772228        nan 0.86444389 0.92855478
 0.92855478 0.928663   0.92987013 0.92857143        nan        nan
 0.90041625        nan 0.8965368  0.94133367 0.94133367 0.93882784
 0.93893606 0.93776224        nan        nan 0.92984515        nan
 0.93633866 0.93763736 0.93763736 0.93114386 0.93253413 0.93630536
        nan        nan 0.93125208        nan 0.93630536 0.93384948
 0.93384948 0.92984515 0.93123543 0.93630536        nan        nan
 0.93363303        nan 0.93630536 0.93363303 0.933633

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which mean

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.87967033
 0.87967033 0.88334998 0.88334998 0.88334998        nan        nan
 0.5               nan 0.5        0.91160506 0.91160506 0.91420246
 0.91

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which mean

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.87485847
 0.87485847 0.87485847 0.87485847 0.87485847        nan        nan
 0.5               nan 0.5        0.89657842 0.89657842 0.9044622
 0.9031635  0.9044622         nan        nan 0.5               nan
 0.5        0.95297203 0.95297203 0.95295538 0.95427073 0.95297203
        nan        nan 0.8719697         nan 0.86590909 0.98973526
 0.98973526 0.98086081 0.98464868 0.98215951        nan        nan
 0.97640693        nan 0.9762987  0.98573094 0.98573094 0.98441558
 0.98441558 0.98203463        nan        nan 0.9762987         nan
 0.97835498 0.97142857 0.97142857 0.97142857 0.979329   0.98073593
        nan        nan 0.96450216        nan 0.97943723 0.96365301
 0.96365301 0.96385281 0.9754329  0.98073593        nan        nan
 0.93701299        nan 0.98073593 0.96235431 0.9623543

...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which mean

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.5               nan 0.5        0.84476357
 0.84476357 0.84474692 0.84604562 0.84474692        nan        nan
 0.5               nan 0.5        0.8702381  0.8702381  0.87420912
 0.87162837 0.87291042        nan        nan 0.5               nan
 0.5        0.89679487 0.89679487 0.89165002 0.89419747 0.89291542
        nan        nan 0.86885614        nan 0.86395271 0.94744422
 0.94744422 0.93452381 0.94110889 0.93712121        nan        nan
 0.94841825        nan 0.94711955 0.95832501 0.95832501 0.952997
 0.95556111 0.95298035        nan        nan 0.95333833        nan
 0.95438728 0.96299534 0.96299534 0.96831502 0.96443556 0.95817516
        nan        nan 0.95775891        nan 0.95687646 0.96033966
 0.96033966 0.96425241 0.96571762 0.95817516        nan        nan
 0.96272061        nan 0.95817516 0.96272061 0.96272061

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which mean

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which mean

Wall time: 5min 4s


In [80]:
print('mean')
print(np.mean(train_scores))
print(np.mean(test_scores))
print("std")
print(np.std(train_scores))
print(np.std(test_scores))
pd.DataFrame(tmp)

mean
0.9447368421052632
0.9154166666666667
std
0.01448919021590396
0.03140517031035212


,0,1,2,3,4,5,6,7
0,"0 0.871854 Name: AUC, dtype: float64","0 0.838291 Name: AUC, dtype: float64","0 0.887872 Name: AUC, dtype: float64","0 0.887872 Name: AUC, dtype: float64","0 0.909611 Name: AUC, dtype: float64","0 0.918383 Name: AUC, dtype: float64","0 0.5 Name: AUC, dtype: float64","0 0.887872 Name: AUC, dtype: float64"
1,"0 0.766667 Name: AUC, dtype: float64","0 0.895833 Name: AUC, dtype: float64","0 0.858333 Name: AUC, dtype: float64","0 0.841667 Name: AUC, dtype: float64","0 0.858333 Name: AUC, dtype: float64","0 0.841667 Name: AUC, dtype: float64","0 0.5 Name: AUC, dtype: float64","0 0.841667 Name: AUC, dtype: float64"
2,"0 0.893593 Name: AUC, dtype: float64","0 0.902365 Name: AUC, dtype: float64","0 0.895118 Name: AUC, dtype: float64","0 0.925629 Name: AUC, dtype: float64","0 0.957666 Name: AUC, dtype: float64","0 0.927155 Name: AUC, dtype: float64","0 0.927155 Name: AUC, dtype: float64","0 0.919908 Name: AUC, dtype: float64"
3,"0 0.841667 Name: AUC, dtype: float64","0 0.9 Name: AUC, dtype: float64","0 0.920833 Name: AUC, dtype: float64","0 0.9625 Name: AUC, dtype: float64","0 0.920833 Name: AUC, dtype: float64","0 0.958333 Name: AUC, dtype: float64","0 0.9 Name: AUC, dtype: float64","0 0.879167 Name: AUC, dtype: float64"
4,"0 0.85431 Name: AUC, dtype: float64","0 0.884821 Name: AUC, dtype: float64","0 0.5 Name: AUC, dtype: float64","0 0.951945 Name: AUC, dtype: float64","0 0.951945 Name: AUC, dtype: float64","0 0.932876 Name: AUC, dtype: float64","0 0.932876 Name: AUC, dtype: float64","0 0.932876 Name: AUC, dtype: float64"
5,"0 0.841667 Name: AUC, dtype: float64","0 0.883333 Name: AUC, dtype: float64","0 0.5 Name: AUC, dtype: float64","0 0.9 Name: AUC, dtype: float64","0 0.879167 Name: AUC, dtype: float64","0 0.866667 Name: AUC, dtype: float64","0 0.904167 Name: AUC, dtype: float64","0 0.883333 Name: AUC, dtype: float64"
6,"0 0.868802 Name: AUC, dtype: float64","0 0.853928 Name: AUC, dtype: float64","0 0.925629 Name: AUC, dtype: float64","0 0.90389 Name: AUC, dtype: float64","0 0.941648 Name: AUC, dtype: float64","0 0.940122 Name: AUC, dtype: float64","0 0.927155 Name: AUC, dtype: float64","0 0.935927 Name: AUC, dtype: float64"
7,"0 0.8 Name: AUC, dtype: float64","0 0.904167 Name: AUC, dtype: float64","0 0.891667 Name: AUC, dtype: float64","0 0.95 Name: AUC, dtype: float64","0 0.9125 Name: AUC, dtype: float64","0 0.8875 Name: AUC, dtype: float64","0 0.870833 Name: AUC, dtype: float64","0 0.870833 Name: AUC, dtype: float64"
8,"0 0.861556 Name: AUC, dtype: float64","0 0.896644 Name: AUC, dtype: float64","0 0.877574 Name: AUC, dtype: float64","0 0.900839 Name: AUC, dtype: float64","0 0.925629 Name: AUC, dtype: float64","0 0.876049 Name: AUC, dtype: float64","0 0.893593 Name: AUC, dtype: float64","0 0.856979 Name: AUC, dtype: float64"
9,"0 0.908333 Name: AUC, dtype: float64","0 0.920833 Name: AUC, dtype: float64","0 0.941667 Name: AUC, dtype: float64","0 0.979167 Name: AUC, dtype: float64","0 0.9625 Name: AUC, dtype: float64","0 0.941667 Name: AUC, dtype: float64","0 0.983333 Name: AUC, dtype: float64","0 0.983333 Name: AUC, dtype: float64"


In [58]:
print(train_scores)
print(test_scores)

[0.9679633867276887, 0.9534706331045004, 0.944698703279939, 0.9328756674294431, 0.9416475972540044, 0.9286803966437834, 0.9591914569031273, 0.9504195270785659, 0.9607170099160945, 0.9664378337147217]
[0    0.858333
Name: AUC, dtype: float64, 0    0.9625
Name: AUC, dtype: float64, 0    0.883333
Name: AUC, dtype: float64, 0    0.929167
Name: AUC, dtype: float64, 0    0.941667
Name: AUC, dtype: float64, 0    0.9125
Name: AUC, dtype: float64, 0    0.920833
Name: AUC, dtype: float64, 0    0.904167
Name: AUC, dtype: float64, 0    0.979167
Name: AUC, dtype: float64, 0    0.908333
Name: AUC, dtype: float64]
